In [3]:
import random


class TaskModule:
    def __init__(self, keywords):
        self.keywords = set(keywords)
        self.state = "INITIAL"


    def handle_request(self, user_input, context):
        raise NotImplementedError("Each task module must implement this method.")

    def match_score(self, user_input):
        input_keywords = set(user_input.lower().split())
        return len(self.keywords & input_keywords) / len(self.keywords)
class WeatherTask(TaskModule):
    def __init__(self):
        super().__init__(["weather", "forecast", "temperature", "rain"])
        self.location_keywords = {"gothenburg", "stockholm", "lund", "paris"}
        self.date_keywords = {"today", "tomorrow"}
        self.detail_keywords = {"temperature", "overall", "rain", "humidity", "wind"}

    def parse_initial_request(self, user_input):
        words = set(user_input.lower().split())
        info = {
            'location': next((word for word in words if word in self.location_keywords), None),
            'date': next((word for word in words if word in self.date_keywords), None),
            'detail': next((word for word in words if word in self.detail_keywords), None)
        }
        return info

    def handle_request(self, user_input, context):
        if self.state == "INITIAL":
            info = self.parse_initial_request(user_input)
            context['weather'] = info

            missing_info = [key for key, value in info.items() if not value]
            if not missing_info:
                self.state = "COMPLETED"
                return f"Here's the {info['detail']} for {info['location']} {info['date']}."
            else:
                self.state = f"ASK_{missing_info[0].upper()}"
                if missing_info[0] == 'location':
                    return "Sure, I can help with that. What's the location? (Options: Gothenburg, Stockholm, Lund, Paris)"
                elif missing_info[0] == 'date':
                    return "Sure, I can help with that. What's the date? (Options: today, tomorrow)"

        else:
            if 'weather' not in context:
                context['weather'] = {}

            if self.state.startswith("ASK_"):
                input_type = self.state.split("_")[1].lower()
                valid_set = getattr(self, f"{input_type}_keywords")
                if user_input.lower() in valid_set:
                    context['weather'][input_type] = user_input.lower()
                    missing_info = [key for key, value in context['weather'].items() if not value]
                else:
                    options = ", ".join(sorted(valid_set))
                    return f"Please provide a valid {input_type}. Options are: {options}."

            if not any(context['weather'].get(k) is None for k in ['location', 'date', 'detail']):
                self.state = "COMPLETED"
                temperature = random.randint(-10, 30)  # Example detail
                return f"Here's the {context['weather']['detail']} for {context['weather']['location']} on {context['weather'].get('date', 'today')}: {temperature}°C."
            else:
                missing_info = [key for key, value in context['weather'].items() if not value]
                if missing_info:
                    self.state = f"ASK_{missing_info[0].upper()}"
                    options = ", ".join(sorted(getattr(self, f"{missing_info[0]}_keywords")))
                    return f"Got it. What's the {missing_info[0]}? (Options: {options})"

class RestaurantTask(TaskModule):
    def __init__(self):
        super().__init__(["restaurant", "food", "eat", "dine", "cuisine"])
        self.type_keywords = {"swedish": ["Meatballs", "Smörgåsbord"], "italian": ["Pizza", "Pasta"], "french": ["Croissant", "Coq au vin"], "japanese": ["Sushi", "Ramen"]}

    def parse_question(self, user_input):
        words = set(user_input.lower().split())
        cuisine_type = next((word for word in words if word in self.type_keywords), None)
        return cuisine_type

    def handle_request(self, user_input, context):
        cuisine_type = self.parse_question(user_input)
        if cuisine_type:
            suggestions = self.type_keywords[cuisine_type]
            suggestion = random.choice(suggestions)
            return f"How about {cuisine_type} food? I recommend trying the {suggestion}."
        else:
            return "What type of food are you in the mood for? (Options: Swedish, Italian, French, Japanese)"

trans_data = {'korsvagen': {'next_tram': 'Number 3 towards Valand at 13:45', 'next_bus': 'RöD at 13:50'},
    'chalmers tvargata': {'next_tram': 'Number 67 towards Utlandagatan at 14:15', 'next_bus': 'Number 33(delayed) at 16:00'},}

class TransportTask(TaskModule):
    def __init__(self):
        super().__init__(["transport", "bus", "tram", "station", "stop"])
        self.state = "INITIAL"

    def handle_request(self, user_input, context):
        stp = next((stp for stp in trans_data if stp.lower() in ' '.join(user_input)), None)

        if stp is None:
            stp_inp = input("Which stop are you at?\n")
            stp_input = stp_inp.lower()
            dest_inp = input("What is your destination?\n")
            dest_input = dest_inp.lower()
            stp = next((stp for stp in trans_data if stp.lower() in stp_input), None)
            #dest = next((dest for dest in trans_data if dest.lower() in dest_input), None)

        if stp is not None:
            trans_info = trans_data[stp]
            self.state = "COMPLETED"
            return f"At {stp.capitalize()}, the next bus is {trans_info['next_bus']} and the next tram is {trans_info['next_tram']}. Enjoy your journey to {dest_inp.capitalize()}. "
        else:
            return f"Sorry, I couldn't find information for the stop {stp_input}."

class TaskRegistry:
    def __init__(self):
        self.tasks = []

    def register_task(self, task_module):
        self.tasks.append(task_module)

    def find_best_task(self, user_input):
        best_score = 0
        best_task = None
        for task in self.tasks:
            score = task.match_score(user_input)
            print(f"Score for {task.__class__.__name__}: {score}")
            if score > best_score:
                best_score = score
                best_task = task
        return best_task

class DigitalAssistant:
    def __init__(self):
        self.context = {}
        self.registry = TaskRegistry()
        self.awaiting_follow_up = False
        self.current_task = None

    def register_task(self, task_module):
        self.registry.register_task(task_module)

    def handle_request(self, request):
        if self.awaiting_follow_up and self.current_task:
            response = self.current_task.handle_request(request, self.context)
            self.awaiting_follow_up = self.current_task.state != "COMPLETED"
        else:
            self.current_task = self.registry.find_best_task(request)
            if self.current_task:
                response = self.current_task.handle_request(request, self.context)
                self.awaiting_follow_up = self.current_task.state != "COMPLETED"
            else:
                response = "I'm sorry, I can't help with that."
        return response

if __name__ == "__main__":
    assistant = DigitalAssistant()
    assistant.register_task(RestaurantTask())
    assistant.register_task(WeatherTask())
    assistant.register_task(TransportTask())

    print("How can I assist you?")
    while True:
        request = input()
        if request.lower() == "quit":
            break
        response = assistant.handle_request(request)
        print(response)
        if not assistant.awaiting_follow_up:
            print("How can I assist you?")

How can I assist you?
How is the weather today
Score for RestaurantTask: 0.0
Score for WeatherTask: 0.25
Score for TransportTask: 0.0
Sure, I can help with that. What's the location? (Options: Gothenburg, Stockholm, Lund, Paris)
Paris
Got it. What's the detail? (Options: humidity, overall, rain, temperature, wind)
overall
Here's the overall for paris on today: 28°C.
How can I assist you?
I need to catch a bus
Score for RestaurantTask: 0.0
Score for WeatherTask: 0.0
Score for TransportTask: 0.2
Which stop are you at?
chalmers tvargata
What is your destination?
Valand
At Chalmers tvargata, the next bus is Number 33(delayed) at 16:00 and the next tram is Number 67 towards Utlandagatan at 14:15. Enjoy your journey to Valand. 
How can I assist you?
I need food suggestions
Score for RestaurantTask: 0.2
Score for WeatherTask: 0.0
Score for TransportTask: 0.0
What type of food are you in the mood for? (Options: Swedish, Italian, French, Japanese)
Swedish
How about swedish food? I recommend try